In [1]:
import pandas as pd
import os

In [2]:
data_dir = '../data/argumentation/'
LOGIC_dir = '../data/LOGIC'

In [3]:
df_cckg = pd.read_csv(os.path.join(data_dir, 'train_cckg.tsv'), sep='\t')
df_iam = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')

In [7]:
examples = pd.read_json(os.path.join(data_dir, 'fallacies_arguments_support.json'))
df_mapping = pd.read_csv(os.path.join(LOGIC_dir, 'mappings.csv'))
df_logic = pd.read_csv(os.path.join(LOGIC_dir, 'train.csv'))

In [8]:
examples.keys()

Index(['faulty generalization', 'false causality', 'circular reasoning',
       'ad populum', 'ad hominem', 'fallacy of logic', 'appeal to emotion',
       'false dilemma', 'equivocation', 'fallacy of extension',
       'fallacy of relevance', 'fallacy of credibility', 'intentional'],
      dtype='object')

In [9]:
def generate_prompt(topic, fallacy_type, supporting=True):
    data = examples[fallacy_type]
    supporting_args = [data['exampleArg1'], data['exampleArg2']]
    fallacies = [data['example1'], data['example2']]
    
     
    text =  f"""You are given a topic.  
    Your task is to generate a {'supporting' if supporting else 'counter'} {fallacy_type} logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    {fallacy_type} fallacy is defined as: {examples[fallacy_type]['definition']}
    examples of {fallacy_type} fallacy are: 
    {fallacies[0]}
    {fallacies[1]}
    
    Here is an example of a supporting {fallacy_type} fallacy: 
    {supporting_args[0]}
    
    return the following using this json format:
    {"{"}
        "topic": {topic},
        "fallacy type": {fallacy_type},
        "{fallacy_type} fallacy support": <> 
    {"}"}
    """
    return text

In [10]:
df_cckg.topic

0                               Cannabis should be legal.
1                          Women should not be in combat.
2       People will use marijuana independent of its l...
3                         women should not partake in war
4                      Marijuana should not be legalized.
                              ...                        
2363      telemarketing should be dictated by the market.
2364    Acting takes children away from their educatio...
2365    Some people's skills are better suited to area...
2366    School prayer allows student to do what they a...
2367    A multi-party system decreases votes of indivi...
Name: topic, Length: 2368, dtype: object

In [11]:
df_cckg[df_cckg.topic=='People should not claim to be gender-neutral.']

,topic,argument,label,stuff
1929,People should not claim to be gender-neutral.,The idea of gender is a construct we invented ...,counter,(redefine; has property; gender-neutral)(gende...


In [12]:
fallacy_types = examples.keys()

for fallacy_type in fallacy_types:
    topic =  df_cckg.topic.sample(1).values[0]
    prompt = generate_prompt(topic=topic, fallacy_type=fallacy_type)
    print(prompt)
    print("######################################\n")

You are given a topic.  
    Your task is to generate a supporting faulty generalization logical fallacy argument in the context of the topic. 
    It should not be longer than 25 words. 
    
    faulty generalization fallacy is defined as: an informal fallacy wherein a conclusion is drawn about all or many instances of a phenomenon on the basis of one or a few instances of that phenomenon. is an example of jumping to conclusions.
    examples of faulty generalization fallacy are: 
    Flight attendants are the nicest people. I should know—I’ve taken two flights.
    The two courses I took at UWI were not very interesting. I don't think its a good university.
    
    Here is an example of a supporting faulty generalization fallacy: 
    {'topic': 'Should Mathematics be a Compulsory Course', 'fallacy type': 'faulty generalization', 'faulty generalization fallacy support': 'Mathematics should be compulsory because everyone benefits from it.'}
    
    return the following using this js

if we ask to generate different supporting fallacy arguments -- can we select those that are most relevant. Say we ask to create 3 supporting arguments, can we say okay, argument 2 is best given the arguments we have on this topic because maybe semantically it's better?

In [29]:
from gensim.models import KeyedVectors
import gensim.downloader as api
try:
    model = KeyedVectors.load('data/glove.model.d2v')
except:
    print("model not found, loading from api")
    model = api.load("glove-wiki-gigaword-50")

model not found, loading from api


In [37]:
df_cckg.head(2)

,topic,argument,label,stuff
0,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,(cannabis; synonym of; marijuana)(legal; cause...
1,Women should not be in combat.,Women and men have the same rights.,counter,(women and men; is a; citizens)(citizens; caus...


In [71]:
import numpy as np
import scipy
import smart_open
import string
topic = 'Adopting gender-neutral language is too costly of a venture.'

def preprocess(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return text.translate(translator)

def get_vector(sentence):
    vec = []
    for word in sentence.split(' '):
        try:
            vec.append(model[word])
        except:
            continue
    return np.mean(vec, axis=0)
    
    
relevant_topics = df_cckg[df_cckg.topic==topic]
vecs = []
for arg in relevant_topics.argument:
    vec = get_vector(preprocess(arg))
    vecs.append(vec)

vecs = np.mean(vecs, axis=0)

In [75]:
def cosine_sim(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

In [78]:
first = "Developing gender-neutral terms requires extensive resources without ensuring effective communication or societal acceptance."
second = "Switching to gender-neutral language poses financial burdens on educational institutions without guaranteed academic improvements."
third = "Enforcing gender-neutral language policies entails excessive costs, ignoring traditional linguistic structures and cultural norms."

first_vec = get_vector(preprocess(third))
cosine_sim(first_vec, vecs)

0.82103854

In [53]:
vecs

[array([ 0.02773985,  0.01690018, -0.00475668,  0.07630099,  0.03978553,
         0.01797122,  0.04636446, -0.10479191, -0.15433212, -0.03494615,
         0.08741698, -0.03494615, -0.07082024], dtype=float32),
 array([ 0.05827828,  0.19919422,  0.08627128,  0.00516041,  0.07045602,
         0.02307627,  0.04220658,  0.03978553,  0.01984975, -0.03279534,
        -0.0037414 ,  0.01690018,  0.08627128, -0.05343712, -0.10039776],
       dtype=float32)]